In [1]:
import os
import pandas as pd
import psycopg2

# Connect to Loop Returns database
conn = psycopg2.connect(
    host=os.environ['LOOP_RETURNS_HOST'],
    port=int(os.environ.get('LOOP_RETURNS_PORT', 5432)),
    database=os.environ['LOOP_RETURNS_DATABASE'],
    user=os.environ['LOOP_RETURNS_USERNAME'],
    password=os.environ['LOOP_RETURNS_PASSWORD']
)

# Explore table schemas
tables = ['loop_return_events', 'loop_merchants', 'loop_monthly_summary']
for table in tables:
    print(f"\n=== {table} ===")
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 3", conn)
    print(df.columns.tolist())
    print(df.head())


=== loop_return_events ===
['event_id', 'return_id', 'merchant_id', 'event_date', 'order_date', 'customer_id', 'product_category', 'item_price_usd', 'quantity', 'return_reason', 'condition_received', 'resolution_type', 'resolution_value_usd', 'exchange_order_value_usd', 'is_test_order', 'is_fraudulent']
      event_id   return_id merchant_id  event_date  order_date customer_id  \
0  EVT-0000001  RET-000001  MERCH-0020  2025-09-15  2025-08-05  CUST-90173   
1  EVT-0000002  RET-000002  MERCH-0022  2025-09-20  2025-08-31  CUST-36685   
2  EVT-0000003  RET-000002  MERCH-0029  2025-09-09  2025-08-23  CUST-82309   

  product_category  item_price_usd  quantity  return_reason  \
0             tops          175.27         1  fit_too_large   
1         footwear          138.72         1  fit_too_small   
2       activewear          151.20         1  quality_issue   

  condition_received     resolution_type  resolution_value_usd  \
0               worn  exchange_same_item                   0.0

/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_35721/3604482673.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 3", conn)
/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_35721/3604482673.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 3", conn)
/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_35721/3604482673.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * 

In [2]:
# Calculate platform-wide average exchange rate
query = """
SELECT 
    ROUND(AVG(exchange_rate_pct), 2) as avg_exchange_rate_pct,
    SUM(exchange_count) as total_exchanges,
    SUM(refund_count) as total_refunds,
    SUM(exchange_count) + SUM(refund_count) as total_resolutions
FROM loop_monthly_summary
"""
avg_exchange = pd.read_sql(query, conn)
print(avg_exchange)

# Calculate weighted average (more accurate - weighted by volume)
query_weighted = """
SELECT 
    ROUND(100.0 * SUM(exchange_count) / NULLIF(SUM(exchange_count) + SUM(refund_count), 0), 2) as weighted_exchange_rate_pct
FROM loop_monthly_summary
"""
weighted_avg = pd.read_sql(query_weighted, conn)
print(f"\nWeighted Exchange Rate: {weighted_avg['weighted_exchange_rate_pct'].iloc[0]}%")

   avg_exchange_rate_pct  total_exchanges  total_refunds  total_resolutions
0                   27.6             4582          11865              16447

Weighted Exchange Rate: 27.86%


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_35721/3324529654.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_exchange = pd.read_sql(query, conn)
/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_35721/3324529654.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weighted_avg = pd.read_sql(query_weighted, conn)


In [2]:
# Re-establish connection
import os
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    host=os.environ['LOOP_RETURNS_HOST'],
    port=int(os.environ.get('LOOP_RETURNS_PORT', 5432)),
    database=os.environ['LOOP_RETURNS_DATABASE'],
    user=os.environ['LOOP_RETURNS_USERNAME'],
    password=os.environ['LOOP_RETURNS_PASSWORD']
)

# Calculate revenue impact per percentage point shift in exchange rate
query = """
SELECT 
    SUM(total_return_value_usd) as total_return_value,
    SUM(revenue_retained_usd) as total_retained,
    SUM(exchange_count) + SUM(refund_count) as total_resolutions,
    SUM(total_return_value_usd) / NULLIF(SUM(exchange_count) + SUM(refund_count), 0) as avg_return_value_per_resolution
FROM loop_monthly_summary
"""
impact_data = pd.read_sql(query, conn)
print(impact_data)

total_return_value = impact_data['total_return_value'].iloc[0]
total_resolutions = impact_data['total_resolutions'].iloc[0]
avg_value_per_resolution = impact_data['avg_return_value_per_resolution'].iloc[0]

# 1 percentage point = 1% of total resolutions shifting from refund to exchange
resolutions_per_pct_point = total_resolutions * 0.01
revenue_per_pct_point = resolutions_per_pct_point * avg_value_per_resolution

print(f"\n=== Revenue Impact per 1% Exchange Rate Increase ===")
print(f"Total resolutions: {total_resolutions:,.0f}")
print(f"Avg return value per resolution: ${avg_value_per_resolution:,.2f}")
print(f"Resolutions shifted per 1pp: {resolutions_per_pct_point:,.0f}")
print(f"💰 Revenue retained per 1pp increase: ${revenue_per_pct_point:,.2f}")

   total_return_value  total_retained  total_resolutions  \
0          4021957.92      1648504.08              16447   

   avg_return_value_per_resolution  
0                       244.540519  

=== Revenue Impact per 1% Exchange Rate Increase ===
Total resolutions: 16,447
Avg return value per resolution: $244.54
Resolutions shifted per 1pp: 164
💰 Revenue retained per 1pp increase: $40,219.58


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_59752/1146958510.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  impact_data = pd.read_sql(query, conn)


In [3]:
# Analyze MoM exchange rate trends
query = """
SELECT 
    month,
    ROUND(100.0 * SUM(exchange_count) / NULLIF(SUM(exchange_count) + SUM(refund_count), 0), 2) as exchange_rate_pct,
    SUM(exchange_count) as exchanges,
    SUM(refund_count) as refunds
FROM loop_monthly_summary
GROUP BY month
ORDER BY month
"""
monthly_trends = pd.read_sql(query, conn)

# Calculate MoM change
monthly_trends['mom_change_pp'] = monthly_trends['exchange_rate_pct'].diff()
monthly_trends['mom_acceleration'] = monthly_trends['mom_change_pp'].diff()

print(monthly_trends.to_string(index=False))

print(f"\n=== Trend Analysis ===")
recent_changes = monthly_trends['mom_change_pp'].dropna().tail(3)
print(f"Last 3 MoM changes (pp): {recent_changes.values}")
print(f"Avg recent MoM change: {recent_changes.mean():.2f}pp")
print(f"Trend: {'Accelerating 📈' if monthly_trends['mom_acceleration'].iloc[-1] > 0 else 'Flattening 📉'}")

  month  exchange_rate_pct  exchanges  refunds  mom_change_pp  mom_acceleration
2025-09              23.61        473     1530            NaN               NaN
2025-10              24.89        542     1636           1.28               NaN
2025-11              27.84        979     2538           2.95              1.67
2025-12              28.79       1209     2991           0.95             -2.00
2026-01              29.68        795     1884           0.89             -0.06
2026-02              31.23        584     1286           1.55              0.66

=== Trend Analysis ===
Last 3 MoM changes (pp): [0.95 0.89 1.55]
Avg recent MoM change: 1.13pp
Trend: Accelerating 📈


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_59752/2094915617.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  monthly_trends = pd.read_sql(query, conn)


In [4]:
# Analyze peak season impact: volume vs exchange rate
query = """
SELECT 
    month,
    SUM(total_return_items) as total_returns,
    SUM(exchange_count) + SUM(refund_count) as total_resolutions,
    ROUND(100.0 * SUM(exchange_count) / NULLIF(SUM(exchange_count) + SUM(refund_count), 0), 2) as exchange_rate_pct,
    SUM(total_return_value_usd) as return_value,
    SUM(revenue_retained_usd) as retained_value
FROM loop_monthly_summary
GROUP BY month
ORDER BY month
"""
seasonal = pd.read_sql(query, conn)

# Mark peak season (Nov-Dec for holiday, Jan for post-holiday returns)
seasonal['is_peak'] = seasonal['month'].isin(['2025-11', '2025-12', '2026-01'])
seasonal['volume_change_pct'] = seasonal['total_resolutions'].pct_change() * 100

print(seasonal[['month', 'total_resolutions', 'volume_change_pct', 'exchange_rate_pct', 'is_peak']].to_string(index=False))

print(f"\n=== Peak vs Non-Peak Comparison ===")
peak = seasonal[seasonal['is_peak']]
non_peak = seasonal[~seasonal['is_peak']]
print(f"Peak season avg exchange rate: {peak['exchange_rate_pct'].mean():.2f}%")
print(f"Non-peak avg exchange rate: {non_peak['exchange_rate_pct'].mean():.2f}%")
print(f"Peak avg volume: {peak['total_resolutions'].mean():,.0f} resolutions/month")
print(f"Non-peak avg volume: {non_peak['total_resolutions'].mean():,.0f} resolutions/month")

  month  total_resolutions  volume_change_pct  exchange_rate_pct  is_peak
2025-09               2003                NaN              23.61    False
2025-10               2178           8.736895              24.89    False
2025-11               3517          61.478421              27.84     True
2025-12               4200          19.419960              28.79     True
2026-01               2679         -36.214286              29.68     True
2026-02               1870         -30.197835              31.23    False

=== Peak vs Non-Peak Comparison ===
Peak season avg exchange rate: 28.77%
Non-peak avg exchange rate: 26.58%
Peak avg volume: 3,465 resolutions/month
Non-peak avg volume: 2,017 resolutions/month


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_59752/354705948.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  seasonal = pd.read_sql(query, conn)
